Data :  https://www.manythings.org/anki/ 

REFERENCE: https://arxiv.org/abs/1409.3215

https://github.com/astorfi/sequence-to-sequence-from-scratch 

https://github.com/pskrunner14/translator

https://github.com/spro/practical-pytorch 


In [1]:
!wget -q https://www.manythings.org/anki/deu-eng.zip
!unzip deu-eng.zip
!mkdir data
!cp -r ./deu.txt ./data/
!mv ./data/deu.txt ./data/eng-deu.txt    #rename to express better what actually is in file

# !rm deu-eng.zip
# import shutil
# shutil.rmtree('./ita_eng')

#just inspect what is in file. You need this because:
#mainly files are too large and take a lot of time to be oppened

with open('./data/eng-deu.txt') as f:
    lines = [line for line in f]
  
print(lines[:1])

Archive:  deu-eng.zip
  inflating: deu.txt                 
  inflating: _about.txt              
['Go.\tGeh.\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8597805 (Roujin)\n']


In [2]:
from __future__ import unicode_literals, print_function, division
import unicodedata
from io import open
import string
import re
import random
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np


import torch
from torch import optim
import torch.nn as nn
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
%matplotlib inline

In [3]:
#@title helpers and dataset

SOS_token = 0  #start of string sos
EOS_token = 1  #end of string eos

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

# Turn a Unicode string to plain ASCII, and make all case independent
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s


#split the file into lines, and then split lines into pairs
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('./data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in [l.split('\t')[0], l.split('\t')[1]]] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs


MAX_LENGTH = 100

###------------------- This part is for simplicity. Do not do it for full scale language translation tasks-----------------

# eng_prefixes = (
#     "i am ", "i m ",
#     "he is", "he s ",
#     "she is", "she s ",
#     "you are", "you re ",
#     "we are", "we re ",
#     "they are", "they re "
# )


# def filterPair(p):
#     return len(p[0].split(' ')) < MAX_LENGTH and \
#         len(p[1].split(' ')) < MAX_LENGTH and \
#         p[1].startswith(eng_prefixes)


# def filterPairs(pairs):
#     return [pair for pair in pairs if filterPair(pair)]
###-------------------------------------------------------------------------------------------------------------------------


def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse=False)
    print("Read %s sentence pairs" % len(pairs))
    # pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)


# class Dataset():
#     """dataset object"""

#     def __init__(self, phase, num_embeddings=None, max_input_length=None, transform=None, auto_encoder=False):
#         """
#         The initialization of the dataset object.
#         :param phase: train/test.
#         :param num_embeddings: The embedding dimentionality.
#         :param max_input_length: The maximum enforced length of the sentences.
#         :param transform: Post processing if necessary.
#         :param auto_encoder: If we are training an autoencoder or not.
#         """
#         if auto_encoder:
#             lang_in = 'eng'
#             lang_out = 'eng'
#         else:
#             lang_in = 'eng'
#             lang_out = 'deu'
#         # Skip and eliminate the sentences with a length larger than max_input_length!
#         input_lang, output_lang, pairs = prepareData(lang_in, lang_out, max_input_length, auto_encoder=auto_encoder, reverse=True)
#         print(random.choice(pairs))

#         # Randomize list
#         random.shuffle(pairs)

#         if phase == 'train':
#             selected_pairs = pairs[0:int(0.8 * len(pairs))]
#         else:
#             selected_pairs = pairs[int(0.8 * len(pairs)):]

#         # Getting the tensors
#         selected_pairs_tensors = [tensorsFromPair(selected_pairs[i], input_lang, output_lang, max_input_length)
#                      for i in range(len(selected_pairs))]

#         self.transform = transform
#         self.num_embeddings = num_embeddings
#         self.max_input_length = max_input_length
#         self.data = selected_pairs_tensors
#         self.input_lang = input_lang
#         self.output_lang = output_lang

#     def langs(self):
#         return self.input_lang, self.output_lang

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):

#         # A tuple which represent a pair
#         pair = self.data[idx]

#         # Define the sample dictionary
#         sample = {'sentence': pair}

#         if self.transform:
#             sample = self.transform(sample)

#         return sample

input_lang, output_lang, pairs = prepareData('eng', 'deu', True)
print(random.choice(pairs))

Reading lines...
Read 255817 sentence pairs
Trimmed to 255817 sentence pairs
Counting words...
Counted words:
eng 16400
deu 36316
['it is very kind of you to help me .', 'es ist sehr nett von dir mir zu helfen .']


In [4]:
#@title model

### Encoder Part
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# ### Simple RNN as Decoder 
# class DecoderRNN(nn.Module):
#     def __init__(self, hidden_size, output_size):
#         super(DecoderRNN, self).__init__()
#         self.hidden_size = hidden_size

#         self.embedding = nn.Embedding(output_size, hidden_size)
#         self.gru = nn.GRU(hidden_size, hidden_size)
#         self.out = nn.Linear(hidden_size, output_size)
#         self.softmax = nn.LogSoftmax(dim=1)

#     def forward(self, input, hidden):
#         output = self.embedding(input).view(1, 1, -1)
#         output = F.relu(output)
#         output, hidden = self.gru(output, hidden)
#         output = self.softmax(self.out(output[0]))
#         return output, hidden

#     def initHidden(self):
#         return torch.zeros(1, 1, self.hidden_size, device=device)


### RNN with soft attention as Decoder
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
#@title training

teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length


def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    beta = float('inf')
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if loss<beta:
          beta=loss
          torch.save(encoder.state_dict(), './encoder.pth')
          torch.save(decoder.state_dict(), './decoder.pth')

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print(iter, iter / n_iters * 100, print_loss_avg)

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

plt.switch_backend('agg')
def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)


hidden_size = 256
encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder, attn_decoder, 75000, print_every=5000)

In [19]:
#@title evaluation

def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]


def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')


In [22]:
saved_encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
saved_decoder = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)
saved_encoder.state_dict(torch.load('./encoder.pth'))
saved_decoder.state_dict(torch.load('./decoder.pth'))

evaluateRandomly(saved_encoder, saved_decoder)

> the scientists gathered data .
= die wissenschaftler sammelten daten .
< uberraschungsgeburtstagsfeier freigeschaltet zeitschrift fuhrungsposten menschenmogliche nachbarlich allerbeste kleider allerbeste bevormundende ramsch ramsch ramsch aktentasche sonntagsarbeit sonntagsarbeit allerbeste herausgesprungen freigehalten rustikaler anlangt anlangt ziehen aufzubrechen verbalisieren kleider todesopfer kleider todesopfer hustens allerbeste weideland kleider todesopfer todesopfer hustens niedriger teilte nachbarlich leichteren essentieller spitzentyp wohnstatte scharpe sonntagsarbeit sonntagsarbeit todesopfer nachbarlich allerbeste allerbeste weideland kleider todesopfer heidelbeermarmelade nachbarlich verbalisieren kleider todesopfer heidelbeermarmelade todesopfer nachbarlich storung heidelbeermarmelade sonntagsarbeit zugunfall allerbeste weideland todesopfer kleider todesopfer lotossitz todesopfer essentieller spitzentyp nachbarlich lehrmeister ergreifen fursorglicher zustimmen kompost 